# Measuring dependency relations' degree of functionality (1.2 updated)

It would be cool if we had a measurment for a relation's degree of functionality, that is, whether it mainly takes functional dependents or content dependents. Here's my suggestion.

My suggestion relies on three assumptions:

- A functional deprel is present in as many languages as possible. This can be measured with the mean probability of a deprel over all languages.
- A functional deprel is also a closed class, meaning the amount of word dependents should be relatively small. This can be measured by the conditional entropy of the probability distribution of all word dependents given the deprel. This is then averaged across all languages.
- A functional deprel has an equal distribution in all languages. This can be measured by the conditional entropy of a deprel given its cross-lingual probability distribution. I'm not sure about this one.

More concisely, this tells us:

### Crosslingual deprel entropy

"It should be equally common in all languages"

### Crosslingual deprel probability

"It should be a common relation in all languages"

### Inverse word entropy

"It should have a small number of word dependents"


## Summary

Inverse word entropy works best alone, while crosslingual deprel entropy and crosslingual deprel probability both have their weaknesses.

In [6]:
# Matplotlib
%matplotlib inline
import matplotlib as mpl
from matplotlib import pylab
import matplotlib.pyplot as plt
pylab.rcParams['figure.figsize'] = (10.0, 7.0)
mpl.style.use('ggplot')
from matplotlib.backends.backend_pgf import FigureCanvasPgf
mpl.backend_bases.register_backend('pgf', FigureCanvasPgf)

# Remove warnings
import warnings
#warnings.filterwarnings('ignore')

import sys, os
from os.path import join
sys.path.append("../tools/")
from collections import defaultdict

# Data management libraries
import pandas as pd
#import seaborn as sns
import scipy as sp
import scipy.stats

# Treebank utils, found in "../tools"
import udeval, udtree, lang_utils, dependency_classes

file_format = "conllu"
train_type = "train"
fine_grained_deprels = False
gold_output_base = "/Users/jimmy/dev/edu/nlp-rod/udeval/resources/universaldependencies1-2/universal-dependencies-1.2/"
file_prefix = "function_content_degree"

gold_files = lang_utils.get_ud_paths(gold_output_base, type_=train_type, format_=file_format, coarse=False)

def word_dependent_entropy(trees):
    dep_word_freqs = defaultdict(lambda : defaultdict(int))
    total = 0
    for tree in trees:
        for word, deprel in zip(tree.words, tree.deprels):
            if not fine_grained_deprels:
                deprel = deprel.split(":")[0]
            dep_word_freqs[deprel][word] += 1
    df = pd.DataFrame(dep_word_freqs).fillna(0)  # frequencies
    df = df.divide(df.sum(axis=0), axis=1)       # word probability conditioned on deprel
    entropy_upper_bound = np.log(len(df.index))
    df = pd.Series(sp.stats.entropy(df), index=df.columns).sort_values()
    return df / entropy_upper_bound  # normalize

def word_dependency_entropy(treebanks):
    lang_entropies = {}
    for lang, treebank in treebanks.items():
        lang_entropies[lang] = word_dependent_entropy(udtree.from_files(treebank))
    return pd.DataFrame(lang_entropies).fillna(0.5).mean(axis=1).sort_values()

def crosslingual_deprel_entropy(treebanks):
    # dep_word_freqs[lang][deprel] = freq
    dep_freqs = defaultdict(lambda : defaultdict(int))
    total = 0
    for lang, treebank in treebanks.items():
        for tree in udtree.from_files(treebank):
            for deprel in tree.deprels:
                if not fine_grained_deprels:
                    deprel = deprel.split(":")[0]
                dep_freqs[lang][deprel] += 1
    df = pd.DataFrame(dep_freqs).fillna(0)  # frequencies
    df = df.divide(df.sum(axis=1), axis=0).transpose()
    entropy_upper_bound = np.log(len(df.index))  # ln num_deprels
    df = pd.Series(sp.stats.entropy(df), index=df.columns) / entropy_upper_bound
    return df.sort_values()

def crosslingual_deprel_probability(treebanks):
    # dep_word_freqs[lang][deprel] = freq
    dep_freqs = defaultdict(lambda : defaultdict(int))
    total = 0
    for lang, treebank in treebanks.items():
        for tree in udtree.from_files(treebank):
            for deprel in tree.deprels:
                if not fine_grained_deprels:
                    deprel = deprel.split(":")[0]
                dep_freqs[lang][deprel] += 1
    df = pd.DataFrame(dep_freqs).fillna(0)  # frequencies
    df = df.divide(df.sum(axis=0), axis=1).mean(axis=1).sort_values()
    return df

In [7]:
#crosslingual_probabilities = crosslingual_deprel_probability(gold_files)
#crosslingual_entropies = crosslingual_deprel_entropy(gold_files)
wde = word_dependency_entropy(gold_files)

## Word dependency entropy

In [8]:
wde.sort_values(ascending=True).to_csv('../data/' + file_prefix + '.wde.csv')
wde.sort_values(ascending=False)

conj          0.790917
nmod          0.782685
amod          0.718383
dobj          0.708311
root          0.707690
acl           0.681165
nsubj         0.677052
advcl         0.649690
ccomp         0.625584
xcomp         0.601194
appos         0.594169
name          0.571315
advmod        0.535234
nsubjpass     0.529459
csubj         0.507887
remnant       0.503055
compound      0.481997
parataxis     0.479552
list          0.477510
nummod        0.475401
iobj          0.475123
reparandum    0.468357
dislocated    0.431588
goeswith      0.415589
foreign       0.412718
vocative      0.373986
dep           0.365133
discourse     0.340948
csubjpass     0.326159
det           0.308098
expl          0.283617
case          0.276961
cop           0.271663
mwe           0.252676
aux           0.248924
auxpass       0.240548
mark          0.220008
punct         0.186544
cc            0.134387
neg           0.060288
dtype: float64

/Users/jimmy/dev/miniconda/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
import seaborn as sns
    
def lang_iwe(treebanks):
    lang_entropies = {}
    for lang, treebank in treebanks.items():
        lang_entropies[lang] = word_dependent_entropy(udtree.from_files(treebank))
    return pd.DataFrame(lang_entropies).fillna(0.5)

lang_entropies = lang_iwe(gold_files)
lang_entropies.to_csv('../data/function_content_degree.individual_languages.wde.csv')
sns.heatmap(lang_entropies)